In [2]:
import calendar
import holidays
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import scipy
import seaborn as sns
import glob
import os

from datetime import datetime
from plotly.subplots import make_subplots
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_pipeline

In [3]:
# !pip install holidays

In [4]:
%%time
path = "citibike_data_19_20"
all_files = glob.glob(os.path.join(path, "*.csv"))

df_nyc = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_nyc.head(3)

CPU times: user 15.2 s, sys: 2.37 s, total: 17.5 s
Wall time: 17.7 s


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1463,2019-03-01 00:00:16.0970,2019-03-01 00:24:39.3880,319.0,Fulton St & Broadway,40.711066,-74.009447,347.0,Greenwich St & W Houston St,40.728846,-74.008591,35618,Subscriber,1989,1
1,285,2019-03-01 00:00:32.3850,2019-03-01 00:05:18.1830,439.0,E 4 St & 2 Ave,40.726281,-73.989780,150.0,E 2 St & Avenue C,40.720874,-73.980858,31113,Subscriber,1980,1
2,686,2019-03-01 00:00:47.7970,2019-03-01 00:12:14.3090,526.0,E 33 St & 5 Ave,40.747659,-73.984907,3474.0,6 Ave & Spring St,40.725256,-74.004121,19617,Subscriber,1987,1


In [5]:
df_nyc.info()
df_nyc["starttime"] = pd.to_datetime(df_nyc["starttime"])
df_nyc = df_nyc[["starttime"]].groupby(by = df_nyc["starttime"].dt.date).count().rename(columns={'starttime':'count'}).reset_index()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6694874 entries, 0 to 6694873
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         float64
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           float64
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(6), int64(4), object(5)
memory usage: 766.2+ MB


In [7]:
df_nyc

,starttime,count
0,2019-01-01,21962
1,2019-01-02,37797
2,2019-01-03,41676
3,2019-01-04,43922
4,2019-01-05,17432
...,...,...
176,2020-03-27,26504
177,2020-03-28,8871
178,2020-03-29,9646
179,2020-03-30,15041


In [8]:
df_nyc.rename(columns = {"starttime": "date"}, inplace=True)
df_nyc.sort_values(by="date", inplace=True)
# Group stray values that got into a wrong month
df_nyc = df_nyc.groupby("date").sum().reset_index()
df_nyc["date"] = pd.to_datetime(df_nyc["date"])
df_nyc.head()

,date,count
0,2019-01-01,21962
1,2019-01-02,37797
2,2019-01-03,41676
3,2019-01-04,43922
4,2019-01-05,17432


In [9]:
df_nyc["Year"] = pd.DatetimeIndex(df_nyc["date"]).year
df_nyc["Month"] = pd.DatetimeIndex(df_nyc["date"]).month
df_nyc["Week"] = pd.DatetimeIndex(df_nyc["date"]).week
df_nyc["Day"] = pd.DatetimeIndex(df_nyc["date"]).day
df_nyc["Day of Week"] = pd.DatetimeIndex(df_nyc["date"]).dayofweek
df_nyc["Day of Year"] = pd.DatetimeIndex(df_nyc["date"]).dayofyear

/tmp/ipykernel_16583/3947180096.py:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df_nyc["Week"] = pd.DatetimeIndex(df_nyc["date"]).week


In [10]:
df_nyc.head(3)

,date,count,Year,Month,Week,Day,Day of Week,Day of Year
0,2019-01-01,21962,2019,1,1,1,1,1
1,2019-01-02,37797,2019,1,1,2,2,2
2,2019-01-03,41676,2019,1,1,3,3,3


In [11]:
d = "data/nyc"


In [17]:
df_nyc.to_parquet('data/nyc/citibike_daily.parquet.gzip',compression='gzip')